In [39]:
import logging
import pandas as pd
import ExtractDrtTrips as edt
import PlansManipulation
import SpacialOperations as sops
import ExtractHubs
import Legs2df
import InterpolateTrips
import GTFSpreprocessing
import createGTFS
import importCells
import joinAndexportGTFS

In [40]:
# Constants
legs_input_path = 'Paris/output_legs.csv' # MATSim output legs csv file
studyarea_input_path = 'Paris/shp/cps.shp' # shapefile .shp for the DRT study area
popArea_input_path = 'Paris/shp/parisPopArea.shp'
gtfs_input_dir = 'Paris/gtfs/' # path to where gtfs.zip is stored

# EPSG code !!ONLY NUMBER!! EPSG:3035 --> 3035
crs_import = 2154 # crs of the raw imported data
crs_working = 2154 # crs used for the processing (can be same as crs_import)

# Toggle for preprocessing steps
# GTFS preprocessing to get hexagonal grid
#      --> needed once, can be skipped if GTFScells.csv file exists
toggle_GTFSprep = False
GTFScells_path = 'Paris/output/GTFScells.csv'

# Simulation & analysis start and end time
# Interpolation will only be done in this time window
start_time = '08:00:00'
end_time = '12:00:00'
# Length of each analysis interval
deltaT = 1 # in hours (h), default 1h

In [41]:
# Setup of the logging module
logging.basicConfig(filename='Paris/output/Plans2GTFS.log',
                    level=logging.DEBUG)
console = logging.StreamHandler()
console.setLevel(logging.INFO)
logging.getLogger('').addHandler(console)

In [42]:
import Legs2df
df = Legs2df.run(legs_input_path)

Importing Trips from output_legs.csv file (Matsim Output)
Importing Trips from output_legs.csv file (Matsim Output)
Importing Trips from output_legs.csv file (Matsim Output)


In [43]:
edt.convertCRS(df,crs_import, crs_working,'start_x', 'start_y','end_x', 'end_y')

Ensuring correct CRS
Ensuring correct CRS
Ensuring correct CRS


In [44]:
edt.constructWktPoint(df, 'start_x', 'start_y', 'origin')
edt.constructWktPoint(df, 'end_x', 'end_y', 'destination')

Converting Coordinates to WKT
Converting Coordinates to WKT
Converting Coordinates to WKT
Converting Coordinates to WKT
Converting Coordinates to WKT
Converting Coordinates to WKT


In [45]:
df = sops.createGeoDataFrame(df['origin'], 'EPSG:' + str(crs_working), df)

In [46]:
df = df.to_crs(crs=crs_working)

In [47]:
DfHubs = ExtractHubs.run(df)

Extracting Hubs based on trip data
Extracting Hubs based on trip data
Extracting Hubs based on trip data


In [48]:
ExtractHubs.assignHub2Trip(DfHubs, df)

In [49]:
DfHubs.to_csv("Paris/output/hubs.csv",sep=';')

In [50]:
if toggle_GTFSprep: GTFSpreprocessing.run(GTFScells_path)

In [51]:
CellsDF = importCells.run(GTFScells_path, crs_working)

Importing hexagonal grid created by Public-Transport-Analysis tool
Importing hexagonal grid created by Public-Transport-Analysis tool
Importing hexagonal grid created by Public-Transport-Analysis tool
adding all waiting and travel times to gtfs: stop_times.txt:  35%|███▍      | 164/469 [1:18:47<2:26:32, 28.83s/it]


In [52]:
CellsDF = sops.removeExcessGrid(CellsDF, studyarea_input_path, crs_working, popArea_input_path, False)

Removing hexagons outside of the regarded study area
Removing hexagons outside of the regarded study area
Removing hexagons outside of the regarded study area


In [53]:
CellsDF = sops.countTripsPerCell(CellsDF, df)

Counting trips originating in each cell of hexagonal grid
Counting trips originating in each cell of hexagonal grid
Counting trips originating in each cell of hexagonal grid


In [54]:
sops.addCellId2Trip(CellsDF,df)

Assigning corresponding cell to origin of trips
Assigning corresponding cell to origin of trips
Assigning corresponding cell to origin of trips


In [55]:
CellsDF.to_csv('Paris/output/cells_v1.csv',sep=';')

In [56]:
df.to_csv('Paris/output/trips.csv',sep=';')

In [57]:
CellsDF = InterpolateTrips.runAllIntervals(df, CellsDF, deltaT, [start_time, end_time], DfHubs)

Starting interpolation of travel and wait times
Starting interpolation of travel and wait times
Starting interpolation of travel and wait times
Interpolation for time interval:   0%|          | 0/4 [00:00<?, ?it/s]


Interval 1
Interpolation for Time interval 8:0:00 to 9:0:00
Hub_21
Found 139 trips to interpolate



100%|██████████| 4/4 [00:00<00:00, 774.71it/s]

100%|██████████| 4/4 [00:00<00:00, 1273.99it/s]


Hub_20
Found 150 trips to interpolate



100%|██████████| 25/25 [00:00<00:00, 1530.66it/s]

100%|██████████| 25/25 [00:00<00:00, 2041.02it/s]


...


100%|██████████| 7/7 [00:00<00:00, 1698.49it/s]

100%|██████████| 7/7 [00:00<00:00, 1210.83it/s]

Hub_24
Found 0 trips to interpolate
Time interval has not enough entries for Hub_24


Hub_26
Found 0 trips to interpolate
Time interval has not enough entries for Hub_26
Hub_27
Found 0 trips to interpolate
Time interval has not enough entries for Hub_27
Hub_23
Found 7 trips to interpolate



100%|██████████| 1/1 [00:00<00:00, 1372.03it/s]

Interpolation for time interval: 100%|██████████| 4/4 [00:08<00:00,  2.17s/it]

Hub_8
Found 0 trips to interpolate
Time interval has not enough entries for Hub_8
Hub_25
Found 0 trips to interpolate
Time interval has not enough entries for Hub_25
Hub_28
Found 0 trips to interpolate
Time interval has not enough entries for Hub_28


In [58]:
drtGTFS = createGTFS.run(CellsDF, DfHubs, deltaT, [start_time, end_time])

adding all stops to gtfs: stops.txt: 100%|██████████| 469/469 [00:00<00:00, 1563.11it/s]
adding all waiting and travel times to gtfs: stop_times.txt:   0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as RGF93 v1 / Lambert-93 (the single non-null crs provided).
  warnings.warn(
adding all waiting and travel times to gtfs: stop_times.txt:  34%|███▍      | 160/469 [00:04<00:09, 34.13it/s]

KeyboardInterrupt: 